In [ ]:
using PyCall
using PyPlot
using Statistics

yf = pyimport("yfinance")
np = pyimport("numpy")

include("OptimPortfolio.jl")
include("Utils.jl")

# Data donwload and basic analysis 

In [ ]:
start = "2013-01-01"
finish = "2017-01-01" 

start_test = "2017-01-01"
finish_test  ="2018-01-01"

assets = ["MSFT", "AAPL", "TSLA", "GOOG", "META", "AMZN"] #"^GSPC"

df = yf.download(assets, start, finish, progress=false)

df_test = yf.download(assets, start_test, finish_test, progress=false)

market = yf.download("^GSPC", start, finish, progress=false)

market_test = yf.download("^GSPC", start_test, finish_test, progress=false)

df_log_ret = (df["Adj Close"] / df["Adj Close"].shift(1)).apply(np.log)
df_test_log_ret = (df_test["Adj Close"] / df_test["Adj Close"].shift(1)).apply(np.log)

market_log_ret = (market["Adj Close"] / market["Adj Close"].shift(1)).apply(np.log)
market_test_log_ret = (market_test["Adj Close"] / market_test["Adj Close"].shift(1)).apply(np.log);

## Asset analysis

In [ ]:
fig, ax = plt.subplot_mosaic("""AB
    CD
    EF""", figsize=(8*2, 6*3))

# Stock prices
df["Adj Close"].plot(ax=ax["A"])

ax["A"].legend(ncol=2)

df_test["Adj Close"].plot(ax=ax["B"])

ax["B"].legend(ncol=2)

#Returns
df["Adj Close"].pct_change().plot(ax=ax["C"])
#df_log_ret.plot(ax=ax["C"])

ax["C"].legend(ncol=2)

df_test["Adj Close"].pct_change().plot(ax=ax["D"])
#df_test_log_ret.plot(ax=ax["D"])

ax["D"].legend(ncol=2)

#Histogram of returns
for asset in assets
    df["Adj Close"][asset].pct_change().hist(ax=ax["E"], bins=50, alpha=0.8, label=asset)
    #df_log_ret[asset].hist(ax=ax["E"], bins=50, alpha=0.8, label=asset)
end

ax["E"].legend(ncol=2)

for asset in assets
    df_test["Adj Close"][asset].pct_change().hist(ax=ax["F"], bins=30, alpha=0.8, label=asset)
    #df_test_log_ret[asset].hist(ax=ax["F"], bins=30, alpha=0.8, label=asset)
end

ax["F"].legend(ncol=2)

## Market analysis 

In [ ]:
fig, ax = plt.subplot_mosaic("""AB
    CD
    EF""", figsize=(8*2, 6*3))

# Stock prices
market["Adj Close"].plot(ax=ax["A"])

ax["A"].legend(ncol=2)

market_test["Adj Close"].plot(ax=ax["B"])

ax["B"].legend(ncol=2)

#Returns
market["Adj Close"].pct_change().plot(ax=ax["C"])
#market_log_ret.plot(ax=ax["C"])

ax["C"].legend(ncol=2)

market_test["Adj Close"].pct_change().plot(ax=ax["D"])
#market_test_log_ret.plot(ax=ax["D"])

ax["D"].legend(ncol=2)

#Histogram of returns
market["Adj Close"].pct_change().hist(ax=ax["E"], bins=50, alpha=0.8, label="S&P500")
#market_log_ret.hist(ax=ax["E"], bins=50, alpha=0.8, label="S&P500")

ax["E"].legend(ncol=2)

market_test["Adj Close"].pct_change().hist(ax=ax["F"], bins=20, alpha=0.8, label="S&P500")
#market_test_log_ret.hist(ax=ax["F"], bins=20, alpha=0.8, label="S&P500")

ax["F"].legend(ncol=2)

# Resample data to 1-month frequency

In [ ]:
freq = "1M"

df = df.resample(freq).mean()

df_test = df_test.resample(freq).mean()

market = market.resample(freq).mean()

market_test = market_test.resample(freq).mean()

df_log_ret = (df["Adj Close"] / df["Adj Close"].shift(1)).apply(np.log)
df_test_log_ret = (df_test["Adj Close"] / df_test["Adj Close"].shift(1)).apply(np.log)

market_log_ret = (market["Adj Close"] / market["Adj Close"].shift(1)).apply(np.log)
market_test_log_ret = (market_test["Adj Close"] / market_test["Adj Close"].shift(1)).apply(np.log);

# Mean-Variance portfolio optimization 

In [ ]:
μ = df["Adj Close"].pct_change().mean().values
Σ = df["Adj Close"].pct_change().cov().values

μ_log = df_log_ret.mean().values
Σ_log = df_log_ret.cov().values

portfolio = Portfolio(µ, Σ)
portfolio_log = Portfolio(μ_log, Σ_log);

### Fixed returns 

In [ ]:
target_return = 0.03

w_opt = MPT_fixed_return(portfolio, target_return)

return_portfolio = sum(w_opt.*μ)
risk_portfolio = sqrt(transpose(w_opt)*portfolio.Σ*w_opt)

return_market = market["Adj Close"].pct_change().mean()
risk_market = sqrt(market["Adj Close"].pct_change().var())

println("Expected Returns Portfolio: ", return_portfolio)
println("Risk Portfolio: ", risk_portfolio)
#println("Weights:", w_opt)

println("\nExpected Return Market: ", return_market)
println("Risk Market: ", risk_market)

println("\nPortfolio-market return ratio: ", return_portfolio/return_market)
println("Portfolio-market risk ratio: ", risk_portfolio/risk_market)

### Fixed risk 

In [ ]:
target_risk = 0.035

w_opt = MPT_fixed_risk(portfolio, target_risk)

return_portfolio = sum(w_opt.*μ)
risk_portfolio = sqrt(transpose(w_opt)*portfolio.Σ*w_opt)

return_market = market["Adj Close"].pct_change().mean()
risk_market = sqrt(market["Adj Close"].pct_change().var())

println("Expected Returns Portfolio: ", return_portfolio)
println("Risk Portfolio: ", risk_portfolio)
#println("Weights:", w_opt)

println("\nExpected Return Market: ", return_market)
println("Risk Market: ", risk_market)

println("\nPortfolio-market return ratio: ", return_portfolio/return_market)
println("Portfolio-market risk ratio: ", risk_portfolio/risk_market)

# Efficient frontier 

In [ ]:
RetRisk, weights = MPT_efficient_frontier(portfolio, 100);

RetRisk_log, weights_log = MPT_efficient_frontier(portfolio_log, 100);

risks = sqrt.(diag(portfolio.Σ))
risks_log = sqrt.(diag(portfolio_log.Σ))

plt.figure(figsize=(8*2, 6*2))

plt.subplot(2, 2, 1)

for i in 1:length(portfolio.μ)

    plt.scatter(risks[i], portfolio.μ[i], s=100, label=assets[i])
    
end

plt.scatter(risk_market, return_market, s=100, label="S&P500")

plt.plot(RetRisk[:, 2], RetRisk[:, 1], color="k", lw=3, ls="--", label="Efficient frontier")

plt.ylabel("Expected return", fontsize=20, labelpad=15)
plt.xlabel("Risk", fontsize=20, labelpad=15)

plt.legend(ncol=2)

plt.subplot(2, 2, 2)

for i in 1:length(portfolio.μ)

    plt.scatter(risks_log[i], portfolio_log.μ[i], s=100, label=assets[i])
    
end

plt.scatter(risk_market, return_market, s=100, label="S&P500")

plt.plot(RetRisk_log[:, 2], RetRisk_log[:, 1], color="k", lw=3, ls="--", label="Efficient frontier")

plt.ylabel("Expected log return", fontsize=20, labelpad=15)
plt.xlabel("Risk", fontsize=20, labelpad=15)

plt.legend(ncol=2)

#
plt.subplot(2,2,3)

N = Int(10^6)

rets = zeros(N)
risks = zeros(N)
sharpe_ratio = zeros(N)

for i in 1 : N

    w = rand(length(portfolio.μ))

    w = w / sum(w)
   
    return_portfolio = sum(w.*portfolio.μ)
    risk_portfolio = sqrt(transpose(w)*portfolio.Σ*w)
    
    rets[i] = return_portfolio
    risks[i] = risk_portfolio
    
    sharpe_ratio[i] = return_portfolio / risk_portfolio
    
end

plt.plot(RetRisk[:, 2], RetRisk[:, 1], color="k", lw=3, ls="--", label="Efficient frontier")

plt.scatter(risks, rets, c=sharpe_ratio, s=8)

plt.ylabel("Expected return", fontsize=20, labelpad=15)
plt.xlabel("Risk", fontsize=20, labelpad=15)

plt.colorbar()

#
plt.subplot(2,2,4)

N = Int(10^6)

rets = zeros(N)
risks = zeros(N)
sharpe_ratio = zeros(N)

for i in 1 : N

    w = rand(length(portfolio_log.μ))

    w = w / sum(w)
   
    return_portfolio = sum(w.*portfolio_log.μ)
    risk_portfolio = sqrt(transpose(w)*portfolio_log.Σ*w)
    
    rets[i] = return_portfolio
    risks[i] = risk_portfolio
    
    sharpe_ratio[i] = return_portfolio / risk_portfolio
    
end

plt.plot(RetRisk_log[:, 2], RetRisk_log[:, 1], color="k", lw=3, ls="--", label="Efficient frontier")

plt.scatter(risks, rets, c=sharpe_ratio, s=8)

plt.ylabel("Expected log return", fontsize=20, labelpad=15)
plt.xlabel("Risk", fontsize=20, labelpad=15)

plt.colorbar()

plt.subplots_adjust(wspace=0.3, hspace=0.3)

# Portfolio testing 

Up to this moment we have optimized the weights of our portfolio for any desired level of returns and risk. However, this is done in a training (in-sample) set, but in the future things could change and our optimal weights could'nt be optimal any more. Here we check how expected returns and expected risk for the optimized weights could vary in the future by applying our optimal weights to a test (out of sample) set. We will show 2 results:

- An example of the out-of-sample performance of the in-sample optimized portfolio yielding a fixed return of 0.025

- The Yield Curve of all in-sample optimal portfolios (the ones on the efficient frontier) in the out-of-sample set.

In [ ]:
#EXAMPLE

#Test in "future" data
df_test_returns = df_test["Adj Close"].pct_change()

target_return = 0.025

#Optimize in past data
w_opt = MPT_fixed_return(portfolio, target_return)

returns_test = df_test_returns.values[2:length(df_test_returns), :] * w_opt
    
risk = round(sqrt(var(returns_test)), digits=3)

dates = [df_test["Adj Close"].index[i] for i in 2: length(df_test)]

mean_return = round(mean(returns_test), digits=4)
return_market = round(market_test["Adj Close"].pct_change().mean(), digits=4)
risk_market = round(sqrt(market_test["Adj Close"].pct_change().var()), digits=3);

#YIELD CURVE
portfolio_test = Portfolio(df_test_returns.mean().values, df_test_returns.cov().values)

target_returns = 0.021 : 0.001 : 0.048

final_risks = []
final_returns = []

expected_risks = []
expected_returns = []

@time for target_return in target_returns

    w_opt = MPT_fixed_return(portfolio, target_return)

    expected_return = sum(w_opt.*portfolio.μ)
    expected_risk = sqrt(transpose(w_opt)*portfolio.Σ*w_opt)

    returns = df_test_returns.values[2:length(df_test_returns), :] * w_opt

    actual_risk = round(sqrt(var(returns)), digits=6)
    actual_mean_return = round(mean(returns), digits=6)
    
    append!(expected_risks, expected_risk)
    append!(expected_returns, expected_return)
    
    append!(final_risks, actual_risk)
    append!(final_returns, actual_mean_return)
    
end

RetRisk_test, weights = MPT_efficient_frontier(portfolio_test, 50);

In [ ]:
fig, ax = plt.subplot_mosaic("AB", figsize=(8*2,6))

ax["A"].plot(dates, returns_test, lw=3, marker="o", ms=12, label="Portfolio simulated returns")

ax["A"].axhline(mean_return, color="k", lw=3, label="Portfolio: μ=$mean_return,  σ=$risk")
ax["A"].axhline(return_market, color="k", lw=3, ls="--", label="Market: μ=$return_market,  σ=$risk_market")

ax["A"].tick_params("x", rotation=45)

ax["A"].legend()


ax["B"].scatter(risk_market, return_market, color="k", s=100, label="S&P500")

ax["B"].plot(expected_risks, expected_returns, marker="o", label="Expected Frontier", color="k")
ax["B"].plot(final_risks, final_returns, marker="o", label="Optimized Portfolios")

ax["B"].plot(RetRisk_test[:, 2], RetRisk_test[:, 1], color="C1", lw=1, ls="-", marker="o")

ax["B"].set_ylabel("Efficient return", fontsize=20, labelpad=15)
ax["B"].set_xlabel("Risk", fontsize=20, labelpad=15)

ax["B"].legend()

#INSET
axins = ax["B"].inset_axes([0.62, 0.1, 0.35, 0.4])

axins.plot(final_risks[1:8], final_returns[1:8], marker="o", color="r")
axins.plot(final_risks[9:end-12], final_returns[9:end-12], marker="o", color="g")
axins.plot(final_risks[end-12:end-7], final_returns[end-12:end-7], marker="o", color="r")

axins.plot(RetRisk_test[end-12:end, 2], RetRisk_test[end-12:end, 1], color="C1", lw=1, ls="-", marker="o")

axins.set_xticks([])
axins.set_yticks([])

ax["B"].indicate_inset_zoom(axins, edgecolor="black")

plt.subplots_adjust(wspace=0.3)

# Genetic Algorithm-based Portfolio Optimization

In [ ]:
μ = df["Adj Close"].pct_change().mean().values
Σ = df["Adj Close"].pct_change().cov().values

portfolio = Portfolio(µ, Σ)

#Efficient frontier using Convex Optimization
RetRisk_MPT, weights_MPT = MPT_efficient_frontier(portfolio, 50)

#Efficient frontier using Evolutionary Optimization

##Define objective function
J(w) = transpose(w)*portfolio.Σ*w

#Iterate through returns to get efficient frontier
target_returns = 0.02 : 0.001 : 0.048 #Input

RetRisk_GA = zeros(length(target_returns), 2)

weights_GA = zeros(length(target_returns), length(portfolio.μ))

@time for i in 1 : length(target_returns)

    ret, risk, weight = EO_fixed_return(J, target_returns[i], portfolio)
    
    RetRisk_GA[i, :] = [ret, risk]
    weights_GA[i, :] = weight
    
end

In [ ]:
plt.plot(RetRisk_MPT[:, 2], RetRisk_MPT[:, 1], color="g", lw=3, zorder=1)
plt.scatter(RetRisk_GA[:, 2], RetRisk_GA[:, 1], color="r")

# TO DO

- Compute MPT with both pct_change and log returns and plot and compare together
- Compute MPT unconstrained